<a href="https://colab.research.google.com/github/1ucky40nc3/TREX/blob/main/TREX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---


# ***Speech Recognition (STT)***


---


In [ ]:
# @title | STT | Installation of Dependencies

%%shell
pip install transformers
pip install ffmpeg-python
pip install -q https://github.com/tugstugi/dl-colab-notebooks/archive/colab_utils.zip

In [ ]:
# @title | STT | Audio Recording Utils
"""Utils for recording audio in a Google Colaboratory notebook.

This code is adapted from:
    https://ricardodeazambuja.com/deep_learning/2019/03/09/audio_and_video_google_colab/
    https://colab.research.google.com/gist/ricardodeazambuja/03ac98c31e87caf284f7b06286ebf7fd/microphone-to-numpy-array-from-your-browser-in-colab.ipynb
"""

SILENT = "&> /dev/null"

import io
import ffmpeg
import numpy as np
from base64 import b64decode
from scipy.io.wavfile import write
from dl_colab_notebooks.audio import audio_bytes_to_np

from IPython.display import display
from IPython.display import HTML
from google.colab.output import eval_js


STYLES_HTML = """
<script>

var styles = `

.output_html {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
}

button {
    width: 300px;
    height: 54px;

    padding: 20px;
    margin: 5px;

    display: flex;
    justify-content: center;
    align-items: center;
    border-radius: 40px;
    border: none;

    text-align: center;
    font-size: 28px;
    
    transition: all 0.5s;
    cursor: pointer;
}

button span {
    display: inline-block;
    position: relative;

    cursor: pointer;
    transition: 0.5s;
}

button span:after {
    content: '🙏';

    position: absolute;
    right: -20px;

    opacity: 0;
    transition: 0.5s;
}

button:hover span {
    padding-right: 25px;
}

button:hover span:after {
    right: 0;
    opacity: 1;
}
`

var styleSheet = document.createElement("style")
styleSheet.type = "text/css"
styleSheet.innerText = styles
document.head.appendChild(styleSheet);

</script>
"""

AUDIO_HTML = """
<script>

var container = document.createElement("div");
var button = document.createElement("button");
var span = document.createElement("span");

button.appendChild(span);
container.appendChild(button);
document.body.appendChild(container);

var base64data = 0;
var reader, recorder, gumStream;

var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
            mimeType : 'audio/webm;codecs=opus'
    };            
    recorder = new MediaRecorder(stream);
    recorder.ondataavailable = function(e) {            
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');

        preview.controls = true;
        preview.src = url;
        container.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data); 
        reader.onloadend = function() {
            base64data = reader.result;
        }
    };
    recorder.start();
};

span.innerText = "⏸︎";
button.style.verticalAlign = "middle";
navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

function toggleRecording() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        span.innerText = "✅"
    }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
    return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve => {
    button.onclick = () => {
        toggleRecording()

        sleep(2000).then(() => {
            resolve(base64data.toString())
        });
    }
});
      
</script>
"""

def record(sample_rate: int = 16000) -> str:
    display(HTML(STYLES_HTML + AUDIO_HTML))
    data = eval_js("data")
    
    audio_bytes = b64decode(data.split(',')[1])
    return audio_bytes_to_np(audio_bytes, sample_rate)

In [ ]:
# @title | STT | Wav2Vec2 Speech Recognition

from typing import List

import torch
import numpy as np

from transformers import Wav2Vec2Tokenizer
from transformers import Wav2Vec2ForCTC


# load model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
wav2vec2 = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

def speech_to_text(audio: np.ndarray) -> List[str]:   
    input_values = tokenizer(
        [audio], 
        return_tensors="pt", 
        padding="longest"
    ).input_values

    logits = wav2vec2(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    text = tokenizer.batch_decode(predicted_ids)
    text = " ".join(text)
    return text


---


# ***Natural Language Processing (NLP)***


---

In [ ]:
# @title | NLP | Install Dependencies

%%shell
pip install transformers
pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html

In [ ]:
# @title | NLP | Initialize the NLP Pipelines

import torch
import transformers


DEVICE = 0 if torch.cuda.is_available() else -1


ZERO_SHOT = transformers.pipeline("zero-shot-classification", device=DEVICE)
TABLE_QA = transformers.pipeline("table-question-answering", model="google/tapas-large-finetuned-wikisql-supervised")

SMALL_TALK = transformers.pipeline("conversational", model="facebook/blenderbot-400M-distill", device=DEVICE)
FEW_SHOT = transformers.pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B', device=DEVICE)

In [66]:
# @title | NLP | Set up Services for Data

import io
import pandas as pd


TRAIN_TABLE = """Location,Train,Start,Destination,Departure,Arrival
Munich,ICE 77,Munich,Berlin,12:30,13:33
Munich,ICE 56,Munich,Leipzig,12:30,13:30
Munich,ICE 33,Leipzig,Berlin,13:30,14:00"""

PLANE_TABLE = """Location,Plane,Start,Destination,Departure,Arrival
Munich,Eurowings (EW8003),Munich,Berlin,12:30,13:33
Munich,Lufthansa (LH1940),Munich,Leipzig,12:30,13:30
Munich,Lufthansa (LH2040),Leipzig,Berlin,13:30,14:00"""

def train_table() -> pd.DataFrame:
    return pd.read_csv(io.StringIO(TRAIN_TABLE))

def plane_table() -> pd.DataFrame:
    return pd.read_csv(io.StringIO(PLANE_TABLE))

def df_to_csv(df) -> str:
    csv = io.StringIO()
    df.to_csv(csv, index=False)
    return csv.getvalue()

def csv_to_df(csv) -> pd.DataFrame:
    return pd.read_csv(io.StringIO(csv))

def travel_tables(tables=[train_table(), plane_table()]) -> pd.DataFrame:
    travel = pd.concat(tables)
    travel = travel.fillna("NONE")

    csv = df_to_csv(travel)
    travel = csv_to_df(csv)
    return travel

travel_samples = """Question: "I am in Hannover. It is 17:48. Which train can I take from Munich to Berlin?" Context: "ICE 33" Answer: "The train ICE 33 will travel from Munich to Berlin."
Question: "I am in Frankfurt. It is 09:32. When will the next flight to Madrid leave?" Context: "07:59" Answer: "The next to Madrid will leave 07:59."
Question: "I am in Hannover. It is 17:48. What is the best train to get to Berlin from London?" Context: "ICE 33" Answer: "The train ICE 33 is the fastest."
Question: "I am in Munich. It is 05:51. From which platform does the RB 61 exit to Zurich?" Context: "8" Answer: "The RB 61 will departs from the platform 8."
Question: "I am in Berlin. It is 12:20. How can I get to London by aircraft?" Context: "Eurowings (EW8003)" Answer: "The flight Eurowings (EW8003) will leave for London."
Question: "I am in London. It is 12:20. I am at a train station. How can I get to Frankfurt?" Context: "ICE 923" Answer: "The train ICE 77 is available for your trip."
Question: "I am in Hamburg. It is 23:43. From which gate does the Lufthansa aircraft (LH1940) take off?" Context: "23" Answer: "The Lufthansa airliner (LH1940) will take off from gate 23."
Question: "I am in Stuttgart. It is 08:24. When will the next ICE to Bremen leave?" Context: "07:59" Answer: "The next to Bremen will leave 07:59."
"""

def travel_time() -> str:
    return "13:21"

def travel_location() -> str:
    return "Munich"

travel_tables()

,Location,Train,Start,Destination,Departure,Arrival,Plane
0,Munich,ICE 77,Munich,Berlin,12:30,13:33,NONE
1,Munich,ICE 56,Munich,Leipzig,12:30,13:30,NONE
2,Munich,ICE 33,Leipzig,Berlin,13:30,14:00,NONE
3,Munich,NONE,Munich,Berlin,12:30,13:33,Eurowings (EW8003)
4,Munich,NONE,Munich,Leipzig,12:30,13:30,Lufthansa (LH1940)
5,Munich,NONE,Leipzig,Berlin,13:30,14:00,Lufthansa (LH2040)


In [ ]:
# @title | NLP | NLP Implementation

from typing import Dict
from typing import List
from typing import Tuple
from typing import Callable
from typing import Optional


class Associations(Dict):
    data: Callable[[], pd.DataFrame]
    samples: str

Response = Tuple[str, Optional[transformers.Conversation]]

Function = Callable[
    [transformers.Conversation], 
    Response
]

class Skill(Dict):
    associations: Associations
    function: Function

class Skills(Dict):
    name: str
    skill: Skill

WarmUp = Callable[
    [transformers.Conversation], 
    transformers.Conversation
]

"""~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Section for Classification on a Zero-Shot basis.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"""

def zero_shot_classification(input: str, 
                             labels: List[str], 
                             top_k: Optional[int] = 1,
                             **kwargs) -> List[str]:
    return ZERO_SHOT(input, labels, **kwargs)["labels"][:top_k]

def skill_classification(input: str, 
                         skills: List[str], 
                         verbose: Optional[bool],
                         **kwargs) -> List[str]:
    if verbose:
        print(f"[DEBUG] |Skill Classification| input: {input}")
        print(f"[DEBUG] |Skill Classification| skills: {skills}")

    skill = zero_shot_classification(input, skills, **kwargs)[0]

    if verbose:
        print(f"[DEBUG] |Skill Classification| skill: {skill}")
    return skill


"""~~~~~~~~~~~~~~~~~~~
Section for Table QA.
~~~~~~~~~~~~~~~~~~~"""

def table_question_answering(input: str, 
                             table: pd.DataFrame, 
                             verbose: Optional[bool] = False, 
                             **kwargs) -> str:
    if verbose:
        print(f"[DEBUG] |Table Question Answering| input: {input}")
        print(f"[DEBUG] |Table Question Answering| table: \n{table}")
    
    output = TABLE_QA(table=table, query=input)

    if verbose:
        print(f"[DEBUG] |Table Question Answering| output: \n{output}")
    return output["answer"]


"""~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Section for Few-Shot Text Generation
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"""

def few_shot(query: str, 
             samples: str, 
             verbose: Optional[bool] = False, 
             **kwargs) -> str:
    if verbose:
        print(f"[DEBUG] |Few-Shot Text Generation| query: \n{query}")
        print(f"[DEBUG] |Few-Shot Text Generation| samples: \n{samples}")
    
    output = FEW_SHOT(samples + query, **kwargs)

    if verbose:
        print(f"[DEBUG] |Few-Shot Text Generation| output: \n{output}")
    return output[0]["generated_text"]


def travel_few_shot(query: str, 
                    samples: str, 
                    verbose: Optional[bool] = False, 
                    **kwargs) -> str:
    if verbose:
        print(f"[DEBUG] |Travel Few-Shot Text Generation| query: \n{query}")
        print(f"[DEBUG] |Travel Few-Shot Text Generation| samples: \n{samples}")

    output = few_shot(query, 
                      samples, 
                      verbose, 
                      **kwargs)
    output = output[len(samples + query):]
    output = output.split('"')[0]

    if verbose:
        print(f"[DEBUG] |Travel Few-Shot Text Generation| output: \n{output}")
    return output


"""~~~~~~~~~~~~~~~~~~~~~~~~~
Section for Skill Functions
~~~~~~~~~~~~~~~~~~~~~~~~~"""

def travel_skill(conversation: transformers.Conversation, 
                 associations: Associations, 
                 verbose: Optional[bool] = False, 
                 **kwargs) -> Response:
    if verbose:
        print(f"[DEBUG] |Travel Skill| input conversation: \n{conversation}")
        print(f"[DEBUG] |Travel Skill| associations: \n{associations}")

    input = conversation.new_user_input
    labels = list(associations.keys())

    variant = zero_shot_classification(input, labels, **kwargs)[0]

    data = associations[variant]["data"]()
    time = associations[variant]["time"]()
    location = associations[variant]["location"]()
    samples = associations[variant]["samples"]
    config = associations[variant]["config"]

    cell = table_question_answering(input, data, verbose)

    input = f"I am in {location}. It is {time}. {input}"
    query = f'Question: "{input}" Context: "{cell}" Answer: "'

    output = travel_few_shot(
        query, 
        samples, 
        verbose, 
        **{**config, **kwargs})

    if verbose:
        print(f"[DEBUG] |Travel Skill| variant: {variant}")
        print(f"[DEBUG] |Travel Skill| cell: {cell}")
        print(f"[DEBUG] |Travel Skill| output: {output}")
    return output, None

def small_talk_skill(conversation: transformers.Conversation, 
                     verbose: Optional[bool] = False, 
                     **kwargs) -> Response:
    if verbose:
        print(f"[DEBUG] |Small Talk Skill| input conversation: \n{conversation}")

    conversation = SMALL_TALK(conversation, **kwargs)
    output = conversation.generated_responses[-1]

    if verbose:
        print(f"[DEBUG] |Small Talk Skill| output conversation: \n{conversation}")
        print(f"[DEBUG] |Small Talk Skill| output: {output}")
    return output, conversation


"""~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Section for Personas and Warm Up
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"""

PERSONAS = [
    "your persona: i am a travel assistant",
]

def warm_up(conversation: transformers.Conversation, 
            personas: List[str] = PERSONAS, 
            verbose: bool = False,
            **kwargs) -> transformers.Conversation:
    for persona in personas:
        conversation.add_user_input(persona)
        conversation = SMALL_TALK(conversation, **kwargs)

    if verbose:
        print(f"[DEBUG] |Warm Up| personas: {personas}")
        print(f"[DEBUG] |Warm Up| personas: {conversation}")
    return conversation



SKILLS: Skills = {
    "small talk": {
        "associations": {}, 
        "function": small_talk_skill
    },
    "travel": {
        "associations": {
            "travel": {
                "data": travel_tables,
                "time": travel_time,
                "location": travel_location,
                "samples": travel_samples,
                "config": {
                    "temperature": 0.6,
                    "do_sample": True,
                    "max_length": 475,
                }
            }
        }, 
        "function": travel_skill
    },
    "other": {
        "associations": {}, 
        "function": small_talk_skill
    },
}

CONFIG = {
    "skills": SKILLS,
    "personas": PERSONAS
} 


class NLP:
    def __init__(self,
                 config: dict = CONFIG,
                 verbose: bool = True,
                 **kwargs):
        self.config = config
        self.skills = CONFIG["skills"]

        self.conversation = transformers.Conversation()
        self.conversation = warm_up(
            self.conversation, 
            personas=config["personas"],
            verbose=verbose,
            **kwargs)

    def __call__(self, input: str, verbose: bool = False, **kwargs):
        if verbose:
            print(f"[DEBUG] |NLP __call__ START|" + "~"*20)
            print(f"[DEBUG] |NLP ATTR skills|: {self.skills}")
            print(f"[DEBUG] |NLP ATTR conversation|: \n{self.conversation}")
            print(f"[DEBUG] |NLP User input|: {input}")

        self.conversation.add_user_input(input)

        skill = skill_classification(
            input, 
            list(self.skills.keys()), 
            verbose=verbose, 
            **kwargs)
        
        pipeline = self.skills[skill]
        function = pipeline["function"]
        associations = pipeline["associations"]
        
        output, conversation = function(
            self.conversation, 
            associations=associations, 
            verbose=verbose,
            **kwargs)

        if conversation:
            self.conversation = conversation

        if verbose:
            print(f"[DEBUG] |NLP User output|: {output}")
            print(f"[DEBUG] |NLP __call__ END|" + "~"*20)
        return output

nlp = NLP()

---


# ***Text-To-Speech (TTS)***


---

In [ ]:
# @title | TTS | Installation of Dependencies

!apt-get install -y espeak

!git clone https://github.com/1ucky40nc3/TransformerTTS.git
%cd TransformerTTS
!git checkout package
!pip install torchaudio
!pip install -r /content/TransformerTTS/requirements.txt
!pip install -r /content/TransformerTTS/TransformerTTS/vocoding/extra_requirements.txt
!python setup.py develop

!wget https://public-asai-dl-models.s3.eu-central-1.amazonaws.com/hifigan.zip
!unzip -q hifigan.zip
!rsync -avq hifigan/ /content/TransformerTTS/TransformerTTS/vocoding/hifigan/

In [ ]:
# @title | TTS | TTS Implementation
%cd /content/TransformerTTS

import torch
import numpy as np
from torchaudio import functional as F

from TransformerTTS.model.factory import tts_ljspeech
from TransformerTTS.vocoding.predictors import HiFiGANPredictor


folder = "/content/TransformerTTS/TransformerTTS/vocoding/hifigan/en"


model, _ = tts_ljspeech()
vocoder = HiFiGANPredictor.from_folder(folder)


def text_to_speech(text: str) -> np.ndarray:
    speech = model.predict(text)
    speech = speech["mel"].numpy().T
    speech = vocoder([speech])[0]

    return speech

%cd ..

def postprocessing(wav: np.ndarray) -> np.ndarray:
    wav = torch.from_numpy(wav)

    wav = wav.unsqueeze(-1).T
    wav = F.apply_codec(
        waveform=wav, 
        sample_rate=22050,
        format="wav", 
        encoding="PCM_F")
    wav = F.resample(
        waveform=wav, 
        orig_freq=22050, 
        new_freq=16000)

    wav = wav.squeeze()
    wav = wav.numpy()
    
    return wav

---


# ***Avatar (PC-AVS)***


---

In [ ]:
# @title | PC-AVS | Install Dependencies

!git clone https://github.com/1ucky40nc3/Talking-Face_PC-AVS.git
%cd /content/Talking-Face_PC-AVS

!pip install -r requirements.txt
!pip install lws
!pip install face-alignment
!pip install av
!pip install torchaudio

!unzip ./misc/Audio_Source.zip -d ./misc/
!unzip ./misc/Input.zip -d ./misc/
!unzip ./misc/Mouth_Source.zip -d ./misc/ 
!unzip ./misc/Pose_Source.zip -d ./misc/

!gdown https://drive.google.com/u/0/uc?id=1Zehr3JLIpzdg2S5zZrhIbpYPKF-4gKU_&export=download
!mkdir checkpoints
!unzip demo.zip -d ./checkpoints/

In [ ]:
# @title | PC-AVS | PC-AVS Implementation

import os
import sys
import torch
import torchvision
from tqdm import tqdm

sys.path.append('..')

from data import create_dataloader
from models import create_model


torch.manual_seed(0)


class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)


def pc_avs_inference(opt, 
                     path_label, 
                     model, 
                     wav) -> str:
    opt.path_label = path_label
    dataloader = create_dataloader(opt, wav=wav)

    fake_image_driven_pose_as = []

    for data_i in tqdm(dataloader):
        _, fake_image_driven_pose_a = model.forward(
            data_i, mode='inference')

        fake_image_driven_pose_as.append(
            fake_image_driven_pose_a)

    filename = os.path.join(
        dataloader.dataset.get_processed_file_savepath(), 
        "G_Pose_Driven_.mp4")

    video_array = torch.cat(fake_image_driven_pose_as, dim=0)
    video_array = video_array.cpu().transpose(1, 3)
    video_array = video_array * 125.5 + 125.5 
    video_array = video_array.type(torch.uint8)
    video_array = torch.rot90(video_array, -1, [1, 2])

    wav = torch.from_numpy(wav)
    wav = torch.unsqueeze(wav, dim=0)
    
    torchvision.io.write_video(
        filename=filename, 
        video_array=video_array,
        fps=25,
        video_codec="libx264",
        audio_array=wav,
        audio_fps=16000,
        audio_codec="aac"
    )    

    del dataloader
    return filename


def avatar(opt,
           path_label,
           wav) -> str:
    opt.isTrain = False

    model = create_model(opt).cuda()
    model.eval()

    return pc_avs_inference(
        opt, 
        path_label, 
        model, 
        wav)
    

opt = Namespace(
    D_input='single', 
    VGGFace_pretrain_path='', 
    aspect_ratio=1.0, 
    audio_nc=256, 
    augment_target=False, 
    batchSize=16, 
    beta1=0.5, 
    beta2=0.999, 
    checkpoints_dir='./checkpoints', 
    clip_len=1, 
    crop=False, 
    crop_len=16, 
    crop_size=224, 
    data_path='/home/SENSETIME/zhouhang1/Downloads/VoxCeleb2/voxceleb2_train.csv', 
    dataset_mode='voxtest', 
    defined_driven=False, 
    dis_feat_rec=False, 
    display_winsize=224, 
    driven_type='face', 
    driving_pose=True, 
    feature_encoded_dim=2560, 
    feature_fusion='concat', 
    filename_tmpl='{:06}.jpg', 
    fitting_iterations=10, 
    frame_interval=1, 
    frame_rate=25, 
    gan_mode='hinge', 
    gen_video=True, 
    generate_from_audio_only=True, 
    generate_interval=1, 
    gpu_ids=[0], 
    has_mask=False, 
    heatmap_size=3, 
    hop_size=160, 
    how_many=1000000, 
    init_type='xavier', 
    init_variance=0.02, 
    input_id_feature=True, 
    input_path='./checkpoints/results/input_path', 
    isTrain=False, 
    label_mask=False, 
    lambda_D=1, 
    lambda_contrastive=100, 
    lambda_crossmodal=1, 
    lambda_feat=10.0, 
    lambda_image=1.0, 
    lambda_rotate_D=0.1, 
    lambda_softmax=1000000, 
    lambda_vgg=10.0, 
    lambda_vggface=5.0, 
    landmark_align=False, 
    landmark_type='min', 
    list_end=1000000, 
    list_num=0, 
    list_start=0, 
    load_from_opt_file=False, 
    load_landmark=False, 
    lr=0.001, 
    lrw_data_path='/home/SENSETIME/zhouhang1/Downloads/VoxCeleb2/voxceleb2_train.csv', 
    max_dataset_size=9223372036854775807, 
    meta_path_vox='./conversations/feaa8fc7-8fc7-4ecf-acef-f06ca221b493/15/avatar.csv', 
    mode='cpu', 
    model='av', 
    multi_gpu=False, 
    nThreads=4, 
    n_mel_T=4, 
    name='demo', 
    ndf=64, 
    nef=16, 
    netA='resseaudio', 
    netA_sync='ressesync', 
    netD='multiscale', 
    netE='fan', 
    netG='modulate', 
    netV='resnext', 
    ngf=64, 
    no_TTUR=False, 
    no_flip=True, 
    no_ganFeat_loss=False, 
    no_gaussian_landmark=False, 
    no_id_loss=False, 
    no_instance=False, 
    no_pairing_check=False, 
    no_spectrogram=False, 
    no_vgg_loss=False, 
    noise_pose=True, 
    norm_A='spectralinstance', 
    norm_D='spectralinstance', 
    norm_E='spectralinstance', 
    norm_G='spectralinstance', 
    num_bins_per_frame=4, 
    num_classes=5830, 
    num_clips=1, 
    num_frames_per_clip=5, 
    num_inputs=1, 
    onnx=False, 
    optimizer='adam', 
    output_nc=3, 
    phase='test', 
    pose_dim=12, 
    positional_encode=False, 
    preprocess_mode='resize_and_crop', 
    results_dir='./conversations/feaa8fc7-8fc7-4ecf-acef-f06ca221b493/15', 
    save_path='./conversations/feaa8fc7-8fc7-4ecf-acef-f06ca221b493/15', 
    serial_batches=False, 
    start_ind=0, 
    style_dim=2560, 
    style_feature_loss=True, 
    target_crop_len=0, 
    train_dis_pose=False, 
    train_recognition=False, 
    train_sync=False, 
    train_word=False, 
    trainer='audio', 
    use_audio=1, 
    use_audio_id=0, 
    use_transformer=False, 
    verbose=False, 
    vgg_face=False, 
    which_epoch='latest', 
    word_loss=False
)

---
---


# ***T-REX*** 🦖


---
---

In [ ]:
# @title | T-REX | Start new Conversation

import uuid
import base64
from IPython.display import HTML


NLP_PERSONA = False # @param {type:"boolean"}
nlp = NLP() if NLP_PERSONA else NLP(config={"skills": SKILLS, "personas": []})

conversation_id = uuid.uuid4()
conversation_dir = f"./conversations/{conversation_id}"
!mkdir ./conversations/
!mkdir {conversation_dir}

interaction_counter = 0
f"Current Conversation is logged at: {conversation_dir}"

!rm -r /content/Talking-Face_PC-AVS/results/id_input_pose_00473_audio_tts_output

In [ ]:
# @title # T-REX 🦖

#@markdown ---
#@markdown ### Default: STT Input

# States if STT output shall be used.
RECORD_AUDIO = False # @param {type:"boolean"}

#@markdown ---
#@markdown ### Alternative: Text Input

# Get the text input.
TEXT_INPUT = "Hello. How are you today?" # @param {type:"string"}

#@markdown ---

VERBOSE = False # @param {type:"boolean"}
verbose = "" if VERBOSE else "&> /dev/null"

interaction_dir = f"{conversation_dir}/{interaction_counter}"
!mkdir {interaction_dir}
interaction_counter += 1


input = speech_to_text(record()) if RECORD_AUDIO else TEXT_INPUT
print(f"[DEBUG] |T-REX STT| Input for NLU: {input}")

input = f'"{input}"'

output = nlp(input, VERBOSE)
output = f'"{output}"'
print(f"[DEBUG] |T-REX NLP| NLP Output: {output}")

audio = text_to_speech(output)
audio = postprocessing(audio)

video = avatar(
    opt,
    "./misc/Input/input 1 ./misc/Pose_Source/00473 158 ./misc/Audio_Source/tts_output.mp3 None 0 None",
    audio
)

# Show the final output.
mp4 = open(video,'rb').read()
data_url = "data:video/mp4;base64," + base64.b64encode(mp4).decode()

HTML("""
<video width=400 controls autoplay>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)